In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
from scipy import stats
from tqdm import tqdm

In [2]:
inputFile = "MAESTRO-d6178bdd-identified_variants_merged_protein_regions-main.tsv"
data = pd.read_csv(inputFile,sep='\t',low_memory=False)

In [22]:
data_processed = data[['Peptide'] + [c for c in data.columns if 'intensity_for_peptide_variant' in c]]

In [23]:
data_processed.replace(0.0,np.nan, inplace = True)

/home/aditya/.local/lib/python3.8/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [25]:
data_processed = data_processed.set_index("Peptide")

In [27]:
data_processed = data_processed.T

In [29]:
data_processed.index = data_processed.index.map(lambda x:'.'.join(x.split('.')[:2]))

In [31]:
def create_label(x):
    if "#Healthy" in x:
        return 1
    elif "#Non-severe-COVID-19" in x:
        return 2
    elif "#Severe-COVID-19" in x:
        return 3
    elif "#Symptomatic-non-COVID-19" in x:
        return 4
    else:
        return 0
data_processed["label"]=data_processed.index.map(lambda x: create_label(x))

In [34]:
data_processed = data_processed[data_processed['label']!=0]

In [64]:
#Filter Data based on NaN for each label
per_label_count = data_processed.groupby('label').count()
filterCondition = (per_label_count >= 18).all()
filter_per_label = per_label_count.loc[:, filterCondition]
filter_column = list(filter_per_label.columns) +['label']
filter_data =  data_processed[filter_column]

In [79]:
for column in tqdm(filter_data):
    if column != "label":
        filter_data[column] = filter_data.groupby("label")[column].transform(lambda x: x.fillna(x.mean()))

  0%|                                                    | 0/90 [00:00<?, ?it/s]/tmp/ipykernel_612715/3972711595.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_data[column] = filter_data.groupby("label")[column].transform(lambda x: x.fillna(x.mean()))
11718it [00:28, 416.98it/s]                                                     


In [82]:
##filter_data.to_csv("filterd_data.tsv", sep="\t")